In [330]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m, WGS84
gcsgp = GCSGeoPandas()
from shared_utils import gtfs_utils_v2, rt_utils

import utils
import update_vars

import altair as alt

In [53]:
import numpy as np

In [2]:
import importlib
importlib.reload(utils)
importlib.reload(update_vars)

<module 'update_vars' from '/home/jovyan/data-analyses/thruway_intercity_bus/update_vars.py'>

# Create utils to get stop sequence and distance info for ridership charts

* currently blocked on warehouse issues for SJJPA feed, could maybe do CC on `gtfs_funnel` cached files...

In [4]:
analysis_date = update_vars.ANALYSIS_DATE

In [5]:
ca = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/ca_boundary.parquet')

## New SJJPA San Joaquins feed

In [6]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [8]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=update_vars.shape_cols)
sanj_shapes = sanj_shapes.to_crs(CA_NAD83Albers_m)
sanj_shapes = sanj_shapes.assign(length_meters = sanj_shapes.geometry.length)

In [70]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=update_vars.trip_cols).query('route_type == "3"')

In [63]:
sanj_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=sanj.feed_key, stop_cols=update_vars.stop_cols).to_crs(CA_NAD83Albers_m)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [11]:
sanj_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=sanj.feed_key, get_df=True,
                                      trip_df = sanj_trips, stop_time_cols=update_vars.stop_time_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [41]:
sanj_shapes_trip_info = (sanj_shapes.merge(sanj_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])
                            .drop(columns=['base64_url', 'regional_feed_type', 'block_id',
                                           'route_desc', 'agency_id', 'network_id',
                                          'route_key'])
                        )

In [42]:
sort_cols = ['route_id', 'direction_id']

In [43]:
def longest_by_route_dir(shape_trip_df):
    #  https://github.com/cal-itp/data-analyses/blob/1abcd5d05163176567f644f6f245366dd9cc53fc/rt_segment_speeds/segment_speed_utils/gtfs_schedule_wrangling.py#L426
    df = (shape_trip_df.query('route_type == "3"')
     .sort_values(sort_cols + ['length_meters'], ascending=[True for i in sort_cols] + [False])
     .drop_duplicates(subset=sort_cols)
     .reset_index(drop=True)
    )
    return df

In [44]:
sanj_long_shapes = longest_by_route_dir(sanj_shapes_trip_info)

In [49]:
# sanj_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

### translate stop_id to ridership data stop_id

In [210]:
bus_stops = bus_st.stop_id.unique()

In [211]:
get_ridership_unique_stops = lambda df: np.union1d(df.orig, df.dest)

In [212]:
rider_stops = get_ridership_unique_stops(ridership)

In [237]:
sanj_bus_to_rider_stops = dict(zip(bus_stops, [x[1:] for x in bus_stops]))
sanj_bus_to_rider_stops['bLOS'] = 'LAX'

In [238]:
# pd.Series(sanj_bus_to_rider_stops.values()).isin(pd.Series(rider_stops)).value_counts()

# gtfs_no_rider_data = pd.Series(sanj_bus_to_rider_stops.values())[~pd.Series(sanj_bus_to_rider_stops.values()).isin(pd.Series(rider_stops))]
# gtfs_no_rider_data #  OK -- all either served by RABA/YARTS or outside CA...

### make chart tables

In [239]:
rt1_test = sanj_long_shapes.query('route_id == "1"')

In [240]:
rt1_test

,feed_key,shape_array_key,geometry,length_meters,trip_instance_key,trip_id,trip_short_name,name,gtfs_dataset_key,service_date,direction_id,route_id,route_type,route_short_name,route_long_name,shape_id
0,d9e1e77d0754b712fc608741ae3836f5,fddc98c94a7294a6a4e4c0c02cb75690,"LINESTRING (265701.090 -584418.173, 265726.924...",404935.670408,5322f590222c5c245f0b996ce8d45de0,5811,AMTthr,Amtrak San Joaquins Schedule,524ea6209600e9a2de34a02cf9068729,2025-09-10,0,1,3,Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - O...,jfy7
1,d9e1e77d0754b712fc608741ae3836f5,dddf1c13185f9f23ab5dea9f85579855,"LINESTRING (89964.411 -293278.458, 89963.886 -...",404265.833098,ba1722adf2f21329a76ec5d85a01890d,5885,AMTthr,Amtrak San Joaquins Schedule,524ea6209600e9a2de34a02cf9068729,2025-09-10,1,1,3,Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - O...,xurp


In [241]:
bus_st = sanj_st.merge(sanj_trips[['trip_id', 'direction_id', 'route_id', 'route_short_name',
                                   'shape_array_key', 'shape_id']], on='trip_id')

In [242]:
bus_st = pd.merge(sanj_stops[['stop_id', 'geometry']], bus_st, on='stop_id')
bus_st = bus_st.assign(amtrak_stop = bus_st.stop_id.map(sanj_bus_to_rider_stops)).sort_values(['trip_id', 'stop_sequence'])

In [243]:
one_dir = bus_st.query('trip_id == @rt1_test.trip_id.iloc[0]')

In [244]:
one_dir.head(3)

,stop_id,geometry,feed_key,trip_id,stop_sequence,arrival_time,departure_time,arrival_sec,departure_sec,arrival_hour,departure_hour,direction_id,route_id,route_short_name,shape_array_key,shape_id,amtrak_stop
49,bSAN,POINT (265701.090 -584418.173),d9e1e77d0754b712fc608741ae3836f5,5811,0,21:25:00,21:25:00,77100,77100,21,21,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,SAN
65,bSOL,POINT (255270.024 -554126.634),d9e1e77d0754b712fc608741ae3836f5,5811,1,21:55:00,21:55:00,78900,78900,21,21,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,SOL
41,bOSD,POINT (244569.780 -532251.690),d9e1e77d0754b712fc608741ae3836f5,5811,2,22:25:00,22:25:00,80700,80700,22,22,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,OSD


In [245]:
import shapely

In [246]:
mi_range = lambda row: np.arange(row.stop_miles, row.next_miles) if not np.isnan(row.stop_miles) and not np.isnan(row.next_miles) else None

In [247]:
one_dir = one_dir.assign(stop_meters = one_dir.geometry.map(lambda x: rt1_test.geometry.iloc[0].project(x)))
one_dir = one_dir.assign(stop_miles = np.round((one_dir.stop_meters / rt_utils.METERS_PER_MILE), 0).astype(int))
one_dir = one_dir.assign(next_miles = one_dir.stop_miles.shift(-1))
one_dir = one_dir.assign(mile_range = one_dir.apply(mi_range, axis=1))

In [248]:
one_dir

,stop_id,geometry,feed_key,trip_id,stop_sequence,arrival_time,departure_time,arrival_sec,departure_sec,arrival_hour,...,direction_id,route_id,route_short_name,shape_array_key,shape_id,amtrak_stop,stop_meters,stop_miles,next_miles,mile_range
49,bSAN,POINT (265701.090 -584418.173),d9e1e77d0754b712fc608741ae3836f5,5811,0,21:25:00,21:25:00,77100,77100,21,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,SAN,0.000000,0,23.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
65,bSOL,POINT (255270.024 -554126.634),d9e1e77d0754b712fc608741ae3836f5,5811,1,21:55:00,21:55:00,78900,78900,21,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,SOL,36806.715578,23,41.0,"[23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30...."
41,bOSD,POINT (244569.780 -532251.690),d9e1e77d0754b712fc608741ae3836f5,5811,2,22:25:00,22:25:00,80700,80700,22,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,OSD,65911.312963,41,84.0,"[41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48...."
27,bIRV,POINT (210370.945 -481730.396),d9e1e77d0754b712fc608741ae3836f5,5811,3,23:10:00,23:10:00,83400,83400,23,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,IRV,134885.642140,84,95.0,"[84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91...."
55,bSNA,POINT (198627.195 -471455.976),d9e1e77d0754b712fc608741ae3836f5,5811,4,23:30:00,23:30:00,84600,84600,23,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,SNA,152473.727729,95,100.0,"[95.0, 96.0, 97.0, 98.0, 99.0]"
1,bANA,POINT (196528.187 -465831.300),d9e1e77d0754b712fc608741ae3836f5,5811,5,23:45:00,23:45:00,85500,85500,23,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,ANA,160749.459470,100,108.0,"[100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106..."
19,bFUL,POINT (192217.436 -458503.526),d9e1e77d0754b712fc608741ae3836f5,5811,6,24:05:00,24:05:00,86700,86700,0,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,FUL,173272.215874,108,134.0,"[108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114..."
538,bLOS,POINT (162725.121 -438368.009),d9e1e77d0754b712fc608741ae3836f5,5811,7,25:00:00,25:00:00,90000,90000,1,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,LAX,214894.253839,134,141.0,"[134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0]"
438,bGLD,POINT (160610.850 -430893.060),d9e1e77d0754b712fc608741ae3836f5,5811,8,25:15:00,25:15:00,90900,90900,1,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,GLD,226278.211794,141,149.0,"[141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147..."
293,bBUR,POINT (151900.275 -423473.936),d9e1e77d0754b712fc608741ae3836f5,5811,9,25:30:00,25:30:00,91800,91800,1,...,0,1,Route 1,fddc98c94a7294a6a4e4c0c02cb75690,jfy7,BUR,240174.094262,149,168.0,"[149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155..."


In [249]:
gtfs_info_df = one_dir[['amtrak_stop', 'stop_sequence','route_id',
         'route_short_name', 'shape_id', 'direction_id',
                        'mile_range']].explode(column='mile_range').rename(columns={'mile_range': 'route_mileage'})

In [250]:
gtfs_info_df

,amtrak_stop,stop_sequence,route_id,route_short_name,shape_id,direction_id,route_mileage
49,SAN,0,1,Route 1,jfy7,0,0.0
49,SAN,0,1,Route 1,jfy7,0,1.0
49,SAN,0,1,Route 1,jfy7,0,2.0
49,SAN,0,1,Route 1,jfy7,0,3.0
49,SAN,0,1,Route 1,jfy7,0,4.0
...,...,...,...,...,...,...,...
314,NHL,10,1,Route 1,jfy7,0,248.0
314,NHL,10,1,Route 1,jfy7,0,249.0
314,NHL,10,1,Route 1,jfy7,0,250.0
314,NHL,10,1,Route 1,jfy7,0,251.0


### add ridership data

In [280]:
source_ridership = pd.read_excel(update_vars.RIDERSHIP_PATH)
source_ridership = source_ridership.assign(od = source_ridership.orig + '->' + source_ridership.dest)
source_ridership = source_ridership.assign(route_short_name = source_ridership.ca_bus_route.str.replace('Rt', 'Route'))
source_ridership = source_ridership.query('trip_year == 2024 & trip_month == 4')

In [281]:
strip_zero = lambda route_str: ' '.join([x.lstrip('0') for x in route_str.split(' ')])
rider_to_gtfs_dict = {'Route 1A': 'Route 1', 'Route 1B': 'Route 1', 'Route 1C': 'Route 1c'}
rider_to_gtfs = lambda route_str: rider_to_gtfs_dict[route_str] if route_str in rider_to_gtfs_dict.keys() else route_str

In [282]:
source_ridership = source_ridership.assign(route_short_name = source_ridership.route_short_name.map(strip_zero).map(rider_to_gtfs))

In [283]:
source_ridership

,ca_bus_route,orig,dest,trip_month,trip_year,ridership,revenue,od,route_short_name
4,Rt 03,ARN,BKY,4,2024,2,41.92,ARN->BKY,Route 3
8,Rt 03,ARN,MTZ,4,2024,2,34.20,ARN->MTZ,Route 3
20,Rt 03,ARN,SAC,4,2024,6,83.40,ARN->SAC,Route 3
27,Rt 03,ARN,SFC,4,2024,2,54.43,ARN->SFC,Route 3
35,Rt 03,BFD,DAV,4,2024,1,33.83,BFD->DAV,Route 3
...,...,...,...,...,...,...,...,...,...
20423,Rt 99,SFC,SKN,4,2024,1,11.16,SFC->SKN,Route 99
20445,Rt 99,SFC,TRU,4,2024,1,134.80,SFC->TRU,Route 99
20452,Rt 99,SKN,SFC,4,2024,4,44.80,SKN->SFC,Route 99
20466,Rt 99,SUI,SFC,4,2024,34,459.66,SUI->SFC,Route 99


In [284]:
df = gtfs_info_df[['amtrak_stop', 'stop_sequence', 'direction_id']].drop_duplicates()

In [285]:
df[['amtrak_stop', 'stop_sequence']].set_index('amtrak_stop').stop_sequence.to_dict()

{'SAN': 0,
 'SOL': 1,
 'OSD': 2,
 'IRV': 3,
 'SNA': 4,
 'ANA': 5,
 'FUL': 6,
 'LAX': 7,
 'GLD': 8,
 'BUR': 9,
 'NHL': 10,
 'BFD': 11}

In [306]:
def stop_sequence_dict_and_direction_id(gtfs_info_df):
    df = gtfs_info_df[['amtrak_stop', 'stop_sequence', 'direction_id']].drop_duplicates()
    assert len(df.direction_id.unique()) == 1
    direction_id = df.direction_id.iloc[0]
    sequence_dict = df[['amtrak_stop', 'stop_sequence']].set_index('amtrak_stop').stop_sequence.to_dict()
    
    return sequence_dict, direction_id

In [307]:
def determine_direction(row, gtfs_info_df):
    
    sequence_dict, direction_id = stop_sequence_dict(gtfs_info_df)
    if sequence_dict[row.orig] < sequence_dict[row.dest]:
        return direction_id
    elif sequence_dict[row.orig] > sequence_dict[row.dest]:
        return 1 - direction_id
    else:
        return ''

In [308]:
sequence_dict, direction_id = stop_sequence_dict_and_direction_id(gtfs_info_df)

In [355]:
rider_one_rt = source_ridership.query('ca_bus_route == "Rt 1A" & orig.isin(@gtfs_info_df.amtrak_stop) & dest.isin(@gtfs_info_df.amtrak_stop)')

In [356]:
rider_one_rt = rider_one_rt.assign(direction_id = rider_one_rt.apply(determine_direction, axis=1, gtfs_info_df = gtfs_info_df),
                orig_seq = rider_one_rt.orig.apply(lambda x: sequence_dict[x]),
                dest_seq = rider_one_rt.dest.apply(lambda x: sequence_dict[x])
                                  )

In [357]:
rider_one_rt.direction_id.value_counts()

0    22
1    19
Name: direction_id, dtype: int64

In [358]:
def running_ridership(df, sequence_dict):

    running_df = []
    for stn in sequence_dict.keys():
        stn_seq = sequence_dict[stn]
        if stn_seq < max(sequence_dict.values()):
            df_at = df.query('orig_seq <= @stn_seq & dest_seq > @stn_seq').assign(departing_station = stn)
            running_df += [df_at]
    return pd.concat(running_df).sort_values('departing_station')

In [359]:
rider_one_rt_dir = rider_one_rt.query('direction_id == 0')

In [360]:
rider_one_rt_dir = running_ridership(rider_one_rt_dir, sequence_dict)

In [361]:
rider_one_rt_dir.ca_bus_route.unique()

array(['Rt 1A'], dtype=object)

In [362]:
gtfs_info_df

,amtrak_stop,stop_sequence,route_id,route_short_name,shape_id,direction_id,route_mileage
49,SAN,0,1,Route 1,jfy7,0,0.0
49,SAN,0,1,Route 1,jfy7,0,1.0
49,SAN,0,1,Route 1,jfy7,0,2.0
49,SAN,0,1,Route 1,jfy7,0,3.0
49,SAN,0,1,Route 1,jfy7,0,4.0
...,...,...,...,...,...,...,...
314,NHL,10,1,Route 1,jfy7,0,248.0
314,NHL,10,1,Route 1,jfy7,0,249.0
314,NHL,10,1,Route 1,jfy7,0,250.0
314,NHL,10,1,Route 1,jfy7,0,251.0


In [363]:
with_distance = rider_one_rt_dir.merge(gtfs_info_df[['amtrak_stop', 'route_mileage']], left_on = 'departing_station', right_on = 'amtrak_stop')

In [364]:
with_distance.route_mileage = with_distance.route_mileage.astype(int)

In [365]:
flow = alt.Chart(with_distance).mark_area().encode(
    alt.X('route_mileage:Q'),
    alt.Y('sum(ridership):Q'),
    color='od',
    tooltip = ['departing_station', 'od', 'ridership']
).properties(width = with_distance.route_mileage.max() * 3)

In [367]:
flow

alt.Chart(...)